# Projet Machine Learning : prédiction de la consommation électrique du bâtiment GreEn-Er.

## 1. Lecture et accès aux données.

In [3]:
import scipy as sp
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def loadData(fileName):
    return pd.read_csv(fileName, index_col=0, parse_dates=True, dtype='float32')

In [5]:
data_folder = Path('data') # les données sont dans le fichier 'data'
csvFile = data_folder / 'conso_globale.csv' # accès au fichier 'conso_globale.csv'
df = loadData(csvFile)

In [6]:
# affichage du DataFrame
#df

## 2. Preprocessing des données.

### 2.1. Données manquantes de température.

In [7]:
# utilisation des données de 'conso_globale' pour avoir accès à la température
# compléter les valeurs manquantes avec les voisins les plus proches (nearest neighbors)
it_na = np.where(df['Temperature'].isna())[0]

# it_na affiche les coordonnées des valeurs manquantes
temp_impute = df['Temperature'].fillna(method='bfill')

# on complète les valeurs manquantes avec .filla et bfill (on utilise la valeur suivante qui est correcte)
np.any(temp_impute.isna())

# df.isna()
# affiche False car il ne manque pas de valeurs

False

In [ ]:
plt.subplots(2,1,figsize=(10,10))
plt.subplot(2,1,1)

plt.plot(temp_impute)
# temp_impute[i_na].plot(subplots=True,color='r')
plt.stem(np.asarray(temp_impute[it_na].index,dtype=object),temp_impute[it_na].values,markerfmt='ro',use_line_collection=True)

# zoom sur 2017
plt.subplot(2,1,2)
# creation d'une variable avec les données de "2017-03-01" à "2017-05-31"
year2017 = temp_impute["2017-03-01":"2017-05-31"]
it_na_2017 = np.where(df['Temperature']["2017-03-01":"2017-05-31"].isna())[0]
plt.plot(year2017)
plt.stem(np.asarray(year2017[it_na_2017].index,dtype=object),year2017[it_na_2017].values,markerfmt='ro',use_line_collection=True)
plt.show()

# en rouge : les données complétées

In [ ]:
# on modifie le DataFrame en le remplaçant par celui complété
df['Temperature'] = temp_impute

### 2.2. Données manquantes de consommation globale.

In [ ]:
# df['Global Consumption'].isna() : contient les consommations globales heures par heures

# df['Global Consumption'].isna()
# affiche les heures et 'False' s'il ne manque pas de données, et 'True' sinon

# ic_na affiche dans un vecteur les index des valeurs manquantes

ic_na = np.where(df['Global Consumption'].isna())[0]

# .fillna permet de compléter des données manquantes
# on utilise la méthode 'bfill' càd que l'on complète les données manquantes avec la donnée prochaine que l'on possède
# on vérifie qu'il n'y a désormais plus de données manquantes

df['Global Consumption'] = df['Global Consumption'].fillna(method='bfill')
conso_impute = df['Global Consumption']
# On vérifie qu'il n'y a désormais plus de données manquantes.

np.any(conso_impute.isna())

# np.any teste s'il y a des élements de temp_impute.isna() qui affichent 'True'
# elle affiche 'False' car il ne manque pas de valeurs. OK!

In [ ]:
plt.figure(figsize=(10,6))
# figsize(largeur, hauteur) en inches par défaut

conso_impute.plot(subplots=True)
# .plot permet de tracer par défaut la consommation énergétique en fonction de la date (heure)
# subplots = True : permet de tracer chaque colonne de conso_impute dans un sous-graphique séparé

plt.stem(np.asarray(conso_impute[ic_na].index,dtype=object),conso_impute[ic_na],markerfmt='ro',use_line_collection=True)

# la fonction stem crée un graphique dans lequel chaque point de données est représenté par une ligne verticale
# qui s'étend de l'axe des x à la valeur y du point
# l'argument 'markerfmt' spécifie comment formater les marqueurs en haut de chaque tige
# dans ce cas, les marqueurs sont des cercles rouges : 'ro'

# conso_impute[ic_na] : contient les coordonnées des données complétées de consommation globale
# conso_impute : contient toutes les données de consommation globale

# use_line_collection=True : une valeur booléenne indiquant s'il faut utiliser un objet 'LineCollection'
# pour dessiner les lignes sur le graphique
# en le définissant à True, cela peut améliorer les performances lorsque l'on trace de grandes quantités de données

plt.show()

### 2.3. Données temporelles manquantes.

##### 2.3.1. On vérifie s'il manque ou non des données temporelles manquantes.

In [ ]:
diff = conso_impute.index - conso_impute.index[0]

# diff : calcule la différence entre chaque valeur de l'index de conso_impute et la première valeur de cet index

# conso_impute.index : renvoie l'ensemble des index de la DataFrame conso_impute

# conso_impute.index[0] : renvoie la première valeur de cet ensemble,
# c'est-à-dire la valeur de l'index située à la première ligne de la DataFrame

# conso_impute.index - conso_impute.index[0] soustrait cette première valeur à chaque valeur de l'index de
# la DataFrame, élément par élément

# le résultat est une série nommée ici diff, qui contient la différence entre chaque index et le premier index de la DataFrame

hours = ( diff.seconds // (3600) + diff.days*24 )

# hours : contient le nombre total d'heures correspondant à chaque différence temporelle de la série diff
# La variable hours est donc une série Pandas de même longueur que diff, contenant des valeurs numériques
# représentant le temps écoulé depuis le début de la période de mesure, exprimé en heures

# diff.seconds renvoie le nombre de secondes dans chaque différence temporelle de diff
# (c'est-à-dire l'extraction de la partie de la différence temporelle qui représente les secondes)

# diff.days renvoie le nombre de jours dans chaque différence temporelle de diff

# diff.days*24 multiplie le nombre de jours dans chaque différence temporelle par 24 (le nombre d'heures
# dans une journée), pour obtenir le nombre total d'heures correspondant à chaque différence temporelle

diff_hours = hours[1:] - hours[:-1]

# diff_hours : contient les différences de temps en heures entre chaque paire d'heures consécutives (soit 1h)

# hours[1:] sélectionne toutes les valeurs de hours sauf la première
# (c'est-à-dire toutes les valeurs à partir de l'indice 1)

# hours[:-1] sélectionne toutes les valeurs de hours sauf la dernière
# (c'est-à-dire toutes les valeurs jusqu'à l'avant-dernier indice)

# hours[1:] - hours[:-1] soustrait à chaque valeur de hours sa valeur précédente, pour obtenir la différence
# de temps entre chaque paire d'heures consécutives dans la série : soit 1h à chaque fois (normalement)

time_gaps = np.where(diff_hours != 1)[0]

# diff_hours != 1 crée un tableau NumPy de booléens avec des 'True' là où les valeurs de diff_hours
# ne sont pas égales à 1 et 'False' sinon

# np.where() trouve les indices des éléments dans le tableau où la valeur est 'True'.
# Dans ce cas, np.where(diff_hours != 1) retourne un tuple contenant un seul élément qui est un vecteur
# contenant les indices où les valeurs de diff_hours ne sont pas égales à 1

# time_gaps est une variable qui reçoit ce tableau NumPy d'indices
# Ces indices représentent les positions dans la série temporelle où se trouvent les "trous" de temps,
# c'est-à-dire les points où il y a des intervalles de temps manquants entre les mesures

print('WARNING! Missing samples:\n{}'.format(conso_impute.index[time_gaps]))

# conso_impute.index[time_gaps] extrait les dates et heures correspondant aux indices des échantillons manquants
# dans la série temporelle

# .format() permet d'insérer la liste des dates et heures manquantes dans le message

# il manque bien des données temporelles que l'on doit combler 

##### 2.3.2. On complète les données manquantes.

In [ ]:
# index où il y a des données manquantes
time_gaps

In [ ]:
# observation de la colonne à partir de laquelle on souhaite compléter nos données
df.iloc[12119]

In [ ]:
df

In [ ]:
# Suppression des lignes avec les index spécifiés (heures non "entières")

# index des lignes à supprimer
positions_to_drop = [24908, 24910, 24911]

# Suppression des lignes
df = df.drop(df.index[positions_to_drop])

#df.index[positions_to_drop] permet d'obtenir la position des lignes à supprimer
#df.drop() : supprime ces lignes du dataframe

In [ ]:
df

In [ ]:
# Ajouter les dates manquantes et les compléter avec la dernière consommation mesurée

row_values = [150.7, 12.2] # valeurs de consommation globale et température
insert_index = 0 # initialisation

date_range = pd.date_range(start="2018-05-21 00:00:00", end="2018-05-27 23:00:00", freq="H")
# plage des dates que l'on souhaite ajouter

# boucle :
for date in date_range:
    insert_row = pd.DataFrame({v: row_values[k] for k, v in enumerate(df.columns)}, index=[date])
    insert_index += 1
    df = df.iloc[:insert_index,].append([insert_row, df.iloc[insert_index:,]])

# Chaque nouvelle ligne de type "DataFrame" est crée en utilisant les valeurs de row_values
# Le dataframe 'df' est mis à jour en utilisant iloc pour diviser le dataframe à l'index 'insert_index'
# et on insère la nouvelle ligne entre les deux parties du DataFrame
#df.iloc() permet de sélectionner un ensemble de lignes connaissant leurs indices
#df.append() permet d'ajouter une ligne à la fin du dataframe

In [ ]:
df

#### 2.3.3. On vérifie qu'il n'y a pas de données manquantes.

In [ ]:
conso = df['Global Consumption']
temp = df['Temperature']

In [ ]:
np.where(conso.isna())

In [ ]:
np.where(temp.isna())

## 3. Redimensionnement des données.

### 3.1. Définition des paramètres du problème.

In [ ]:
nsample = len(conso)
# nombre total d'échantillons (ie. de données de consommation que l'on possède)

# Sélection de la valeur passée à l'instant H
num_days = 14 # number of days (avant changement = 7)
# Nombre de jours utilisés pour l'historique de la prédiction
# On veut utiliser les données des 2 semaines précédentes

history = num_days * 24 + 1
# history : représente le nombre total d'heures d'historiques utilisées pour la prédiction
# L'historique commence à l'instant H de la journée D et remonte jusqu'à l'instant H de la journée D - 7

# Prédiction à l'instant H + prévision (forecast)
forecast = 24*7 # 24 h = 1 day ahead forecasting (avant changement = 24)

# forecast : nombre d'heures d'avance pour lesquels on souhaite effectuer la prédiction

### 3.2. Redimensionnement des données.

In [ ]:
# Récupération de la consommation à l'instant H+forecast (H : heure)
ya_target = conso[history+forecast-1:]

# Extraction de la consommation d'énergie à partir de l'indice 'history+forecast-1' jusqu'à la fin
# Ce sont les données que l'on souhaite prédire

# Récupération des températures à l'instant H (+ éventuellement les valeurs passées)
xa_temp = temp[history - 1 : nsample - forecast] # temp_imput voir plus haut (données de température)
# On récupère les données de températures à partir de l'indice 'history-1' jusqu'à 'nsample-forecast',
# ce qui correspond à la plage de temps où l'on dispose de données de température correspondantes

# Récupération des valeurs passées de la consommation énergétique
Xa = np.zeros( shape = (nsample-forecast-history+1, history) )
for h in range(nsample - forecast - history + 1 ):
    # Take past values
    Xa[h,:] = conso[h : h + history]

# On remplit chacune ligne avec les valeurs passées de consommation d'énergie

# Inversion des variables prédictives pour obtenir la valeur actuelle en première colonne
Xa = np.fliplr(Xa)  # fliplr : "retourner" le vecteur

# Ajout de la température actuelle comme une caractéristique et concaténation à la dernière colonne de X
Xa = np.concatenate( (Xa, xa_temp.to_numpy().reshape(-1, 1) ) ,axis = 1  )
# to_numpy : convertie la donnée en un array numpy
# Xa contient la consommation à un temps donnée (1e colonne) et la température (2e colonne) à ce temps donné


In [ ]:
# Vérification que les données x et y sont correctement enregistrées dans le vecteur temps
print('starting time')
print('{} (x_temp)'.format(xa_temp.index[0]))
print('{} (current predictor X)'.format(conso[0:0+history].index[-1]))
print('{} (last predictor X)'.format(conso[0:0+history].index[0]))
print('{} (target y)'.format(ya_target.index[0]))

print('ending time')
print('{} (x_temp)'.format(xa_temp.index[-1]))
print('{} (current predictor X)'.format(
    conso_impute[(nsample-forecast-history):(nsample-forecast-history)+history].index[-1]))
print('{} (last predictor X)'.format(
    conso_impute[(nsample-forecast-history):(nsample-forecast-history)+history].index[0]))
print('{} (target y)'.format(ya_target.index[-1]))

## 4. Machine learning.

### 4.1. Mettre les données à l'échelle (scaling).

In [ ]:
from sklearn.preprocessing import StandardScaler

# Mise à l'échelle (standardiser) les données
sc = StandardScaler()
Xa_s= sc.fit_transform(Xa)
#StandardScaler() : Standardise les datas en enlevant la valeur moyenne.
#fit_transform(Xa) : Calcul la moyenne et la norme pour transformer les data Xa en format compatible.

### 4.2. L'algorithme Random Forest.

#### 4.2.1. Influence

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import  median_absolute_error

In [ ]:
X = Xa_s #data d'entrainement
y = ya_target #valeur cible

max_depth = 15 # définition de la pronfondeur maximale de l'arbre

regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
                                random_state=0, oob_score=True,
                                verbose=1,
                                criterion='squared_error', # Rk: too time consumming with the more robust 'mae'
                                n_jobs=-1)

#Paramétrisation du modéle de prédiction de "Random forest". On initialise les différents paramétres de la fonction
#pour pouvoir réaliser la régression de nos données.
# n_estimator : définit le nombre d'arbre a créer dans la forêt
# max_depth : définit le nombre maximale de noeuds par arbres
# random_state : choix du générateur de nombre aléatoire
# oob_scores : si true permet de calculer les scores du dataset d'entrainement
# verbose : permet d'avoir plus d'information sur le procéssus de construction (bandeau rouge exécution)
# criterion : calcules de la qualité (choix squarred error)
# n_jobs : nombre de travaux réalisé en parralléles sur le processeur (-1 = maximum)

regr_rf.fit(X,y) #réalise une forêt d'arbres via les données d'entrainement X et y la valeur cible.


In [ ]:
y_rf_a= regr_rf.predict(X) #prédits de nouvelles données y_rf_a cibles selon X (via le modéle regr_rf.fit(X,y) )
r2_train = r2_score(y_rf_a,y) #calcul du coefficient de détermination R2 pour évaluer la méthode (valeur max = 1.0)
mad_train = median_absolute_error(y_rf_a,y) #Calcul de l'erreur medianne absolue (meilleur valeur 0.0)
print('training error r2 = {}, MAD = {}'.format(r2_train,mad_train) ) #impressions de R2 et de l'erreur medianne absolue
print('out of bag score (r2 estimaeted on non used data)= {}'.format(regr_rf.oob_score_)) # impression de R2 sur les données non estimé

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(ya_target.values,'r')
plt.plot(y_rf_a)
plt.legend(['actual','pred'])
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.bar(np.arange(len(regr_rf.feature_importances_)), regr_rf.feature_importances_) #affichage de l'influence des données utilisés
plt.xticks(np.arange(0,24*14+1,24),labels=['D0','D-1','D-2','D-3','D-4','D-5','D-6', 'D-7','D-8','D-9','D-10','D-11','D-12','D-13','Temp'] )

In [ ]:
# Validation croisée
ybis = conso_impute[history+forecast-1:]
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneGroupOut

groups = np.zeros(y.shape)
years = ybis.index.year.to_numpy()
for iy , year in enumerate(np.arange( years.min(),  years.max()+1 )):
    gindex = np.where( years == year)[0]
    groups[gindex] = iy

cv = LeaveOneGroupOut()
scoring = {'r2': 'r2', 'MAD': 'neg_median_absolute_error'}
scores = cross_validate(regr_rf, X, y, cv=cv.split(X,y, groups),
                        scoring = scoring)


mad_test = np.mean(scores['test_MAD'])
r2_test = np.mean(scores['test_r2'])
print('test error r2 = {}, MAD = {}'.format(r2_test,mad_test) )
print('out of bag score (r2 estimaeted on non used data)= {}'.format(regr_rf.oob_score_))

#### 4.2.2. Prédiction.

In [ ]:
# Division des données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=42)

max_depth = 50 # définition de la pronfondeur maximale de l'arbre

regr_rf = RandomForestRegressor(n_estimators=500, max_depth=max_depth,
                                random_state=0, oob_score=True,
                                verbose=1,
                                criterion='squared_error',
                                n_jobs=-1)

regr_rf.fit(X_train,y_train) # réalisation une forêt d'arbres via les données d'entrainement X et y la valeur cible.


In [ ]:
y_rf_a= regr_rf.predict(X_test) #prédits de nouvelles données y_rf_a cibles selon X (via le modéle regr_rf.fit(X,y) )
r2_train = r2_score(y_rf_a,y_test) #calcul du coefficient de détermination R2 pour évaluer la méthode (valeur max = 1.0)
mad_train = median_absolute_error(y_rf_a,y_test) #Calcul de l'erreur medianne absolue (meilleur valeur 0.0)
print('training error r2 = {}, MAD = {}'.format(r2_train,mad_train) ) #impressions de R2 et de l'erreur medianne absolue
print('out of bag score (r2 estimaeted on non used data)= {}'.format(regr_rf.oob_score_)) # impression de R2 sur les données non estimé

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(y_test.values,'r')
plt.plot(y_rf_a)
plt.legend(['actual','pred'])
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.bar(np.arange(len(regr_rf.feature_importances_)), regr_rf.feature_importances_) #affichage de l'influence des données utilisés
plt.xticks(np.arange(0,24*14+1,24),labels=['D0','D-1','D-2','D-3','D-4','D-5','D-6', 'D-7','D-8','D-9','D-10','D-11','D-12','D-13','Temp'] )

In [ ]:
#Cross validation
ybis = conso_impute[history+forecast-1:]
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneGroupOut

groups = np.zeros(y.shape)
years = ybis.index.year.to_numpy()
for iy , year in enumerate(np.arange( years.min(),  years.max()+1 )):
    gindex = np.where( years == year)[0]
    groups[gindex] = iy
cv = LeaveOneGroupOut()
scoring = {'r2': 'r2', 'MAD': 'neg_median_absolute_error'}
scores = cross_validate(regr_rf, X, y, cv=cv.split(X,y, groups),
                        scoring = scoring)


mad_test = np.mean(scores['test_MAD'])
r2_test = np.mean(scores['test_r2'])
print('test error r2 = {}, MAD = {}'.format(r2_test,mad_test) )
print('out of bag score (r2 estimaeted on non used data)= {}'.format(regr_rf.oob_score_))

### 4.3. L'algorithme de régression polynomiale.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import median_absolute_error,r2_score
from sklearn.model_selection import train_test_split

In [ ]:
#X=temp.to_numpy().reshape((-1,1))
X1=Xa_s[:,-1] 

# Les Caractéristiques choisis -- Les température
X=X1.reshape((-1,1))
# On utilise .reshape((-1,1))pour redimensionner un tableau X1 en un tableau à deux dimensions (X) avec une seule
# colonne. C'est la méthode de la classe NumPy ndarray utilisée pour remodeler le tableau. L'argument (-1, 1)
# spécifie la nouvelle forme du tableau.
# La valeur -1, il indique à NumPy d'inférer automatiquement la taille de cette dimension en fonction des autres
# dimensions et de la taille totale du tableau. Ainsi, NumPy déterminera la taille appropriée pour cette dimension
# afin de préserver le nombre total d'éléments du tableau.
# La valeur 1 --> le tableau résultant doit avoir une seule colonne.
# print(X)

y=ya_target 
# Les valeurs cibles que l'on souhaite prédire 

In [ ]:
# Transformation des caractéristiques en utilisant un degré polynomial
degree=6  

# Degré du polynôme --> fait référence à l'ordre du polynôme utilisé pour modéliser la relation entre
# (caractéristiques) et (cible). Le degré du polynôme détermine le nombre maximal de termes polynomiaux inclus
# dans le modèle. Par exemple, un polynôme de degré 1 est une simple régression linéaire, tandis qu'un polynôme
# de degré 2 est une régression quadratique...
# Un degré plus élevé permet au modèle de mieux s'adapter aux données, mais il peut également conduire à un
# surajustement si le degré est choisi de manière excessive par rapport à la complexité des données.

poly_features = PolynomialFeatures(degree=degree, include_bias=False)
# utilisée pour générer de nouvelles caractéristiques en ajoutant des polynômes des caractéristiques d'origine.
# permet d'effectuer une régression polynomiale.

X_poly = poly_features.fit_transform(X)
# pour transformer les caractéristiques d'origine en des polynômes de degré spécifié à l'aide de la méthode
# fit_transform().
# ce qui peut permettre de modéliser des relations plus complexes entre les caractéristiques et la variable cible.

In [ ]:
# Division des données en ensembles d'entraînement et de test
# à l'aide d'une proportion de 80% pour l'ensemble d'entraînement et 20% pour l'ensemble de test.
train_size = int(0.8 * nsample)
X_train, X_test = X_poly[:train_size], X_poly[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# Création du pipeline et Entraînement du modèle sur l'ensemble d'entraînement
linear_regression = LinearRegression()
Modele = Pipeline([("polynomial_features", poly_features),
                         ("linear_regression", linear_regression)])
# un modèle de régression linéaire utilisant la régression polynomiale en utilisant le pipeline.
# Le pipeline lie lie l'objet 'poly_features' et 'linear_regression'

Modele.fit(X_train, y_train)
# --> ajuste le modèle sur les données d'entraînement
# Après l'exécution, on obtient le modèle entraîné Modele qui incorpore à la fois la transformation des
# caractéristiques en polynômes de degré spécifié et la régression linéaire pour prédire les valeurs cibles.

In [ ]:
# Prédiction
y_pred = Modele.predict(X_test)
# Prédiction des valeurs cibles correspondantes aux caractéristiques de l'ensemble de test (X_test)
y_pred

In [ ]:
# Évaluation du modèle
mad = median_absolute_error(y_test, y_pred) # Mean Absolute Error
print("MAD:", mad)
r2 = r2_score(y_test, y_pred) # le coefficient de détermination
print("R²:", abs(r2))
scores = cross_val_score(Modele, X_poly, y,
                             scoring="neg_mean_squared_error", cv=10)
#print(scores)

In [ ]:
t=np.arange(len(X_test))
plt.figure(facecolor='w')
plt.figure(figsize=(15,10))
plt.plot(t, y_test, 'r-', linewidth=2, label='Actual')
plt.plot(t, y_pred, 'b-', linewidth=2, label='Predict')
plt.legend(loc = 'lower right')
plt.title
plt.grid(b=True)
plt.show()

In [8]:
# Validation croisée du modèle pour évaluer ses performances
from sklearn.model_selection import cross_val_score

scores=cross_val_score(Modele, X_poly, y, cv=5,scoring='neg_mean_absolute_error') 

NameError: name 'Modele' is not defined